In [1]:
import os
import numpy as np
import rasterio
from rasterio.features import shapes
from shapely.geometry import shape
import geopandas as gpd

In [11]:
# Set folder path
input_folder = 'RCNN_predicted_masks'
output_folder = 'RCNN_polygonized_outputs'
os.makedirs(output_folder, exist_ok=True)

In [ ]:
for filename in os.listdir(input_folder):
    if filename.endswith(".tif"):
        file_path = os.path.join(input_folder, filename)
        print(f"Processing: {filename}")

        # Read the raster
        with rasterio.open(file_path) as src:
            image = src.read(1)
            transform = src.transform
            crs = src.crs  # get the CRS from the source file

        # Create a mask for polygonizing
        mask = image > 0  # buildings are marked as non-zero

        # Generate shapes (polygons) from the mask
        results = (
            {"properties": {"value": v}, "geometry": s}
            for s, v in shapes(image, mask=mask, transform=transform)
            if v > 0
        )

        # Convert to a GeoDataFrame
        geoms = []
        for res in results:
            geom = shape(res["geometry"])
            geoms.append({"geometry": geom, "value": res["properties"]["value"]})

        gdf = gpd.GeoDataFrame(geoms)
        gdf.set_crs(crs, inplace=True)  # Set the CRS

        # Save to Shapefile
        shp_name = os.path.splitext(filename)[0] + '.shp'
        shp_path = os.path.join(output_folder, shp_name)
        gdf.to_file(shp_path, driver='ESRI Shapefile')
        print(f"Saved shapefile to: {shp_path}")

Processing: Dataset.13_predicted.tif
Saved shapefile to: RCNN_polygonized_outputs\Dataset.13_predicted.shp
Processing: Dataset.25_predicted.tif
Saved shapefile to: RCNN_polygonized_outputs\Dataset.25_predicted.shp
Processing: Dataset.27_predicted.tif
Saved shapefile to: RCNN_polygonized_outputs\Dataset.27_predicted.shp
Processing: Dataset.3_predicted.tif
Saved shapefile to: RCNN_polygonized_outputs\Dataset.3_predicted.shp
Processing: Dataset.41_predicted.tif
Saved shapefile to: RCNN_polygonized_outputs\Dataset.41_predicted.shp
Processing: Dataset.51_predicted.tif
Saved shapefile to: RCNN_polygonized_outputs\Dataset.51_predicted.shp
Processing: Dataset.54_predicted.tif
Saved shapefile to: RCNN_polygonized_outputs\Dataset.54_predicted.shp
Processing: Dataset.8_predicted.tif
Saved shapefile to: RCNN_polygonized_outputs\Dataset.8_predicted.shp
